# MNIST Classification

by learning how to classify the handwritten digits found in the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset!


In Torch, MNIST is conveniently available from the `mnist` package which we will import using

In [1]:
mnist = require 'mnist'
print(mnist)

{
  testdataset : function: 0x0a3bd130
  traindataset : function: 0x0a3bd0b8
}


The functions that were imported can be called to yield the images and labels:

In [2]:
trainData = mnist.traindataset()
print(trainData)

{
  data : ByteTensor - size: 60000x28x28
  size : 60000
  label : ByteTensor - size: 60000
}


which look like

In [4]:
images = {}
labels = {}
for i=1,5 do
    r = torch.random(trainData.size)
    images[i] = trainData.data[r]
    labels[i] = trainData.label[r]
end
itorch.image(images)
for i=1,5 do io.write(labels[i]..' ') end

4 7 4 7 4 

Seems reasonable. Let's see if our computers agree! 

## DataLoader

For the remainder of this workshop, you'll be getting your hands dirty by first filling in a very basic skeleton of the data loader and then writing and tuning several models to get the best performance on MNIST!

### One-sample, two-sample, $n$-sample, batch!

A neural network is like any other function: it takes some inputs (data) and produces some outputs (predictions). In this section, we'll work with the code found in [`mnist/dataloder.lua`](../edit/mnist/dataloader.lua).

Let's start with a quick tour:

The first new piece of machinery is on line 4: 

```lua
local DataLoader = torch.class('DataLoader')
```

If you hearken back to the Intro to Lua segment, you'll recall that Lua tables can be used like classes in standard object-oriented programming. Torch makes this easy by handling all of the boilerplate in [torch.class](https://github.com/torch/torch7/blob/master/doc/utility.md#torch.class) so that all you need to do is implement the methods you care about. In this case, we will want

* `__init` (a default method) which is analagous to `__init__` in Python,
* `run` (a custom method) which will return an iterator over our dataset, and
* `size` (a custom method) which will return the size of the training or validation set


In [6]:
dofile('mnist/dataloader.lua')
local loader = DataLoader{batchSize=4}
iter = loader:run('train')
local n,data = iter()
print(data.labels)
itorch.image(data.images)

  8
  8
 10
 10
[torch.ByteTensor of size 4]



## `nn`: For (Almost) All of Your Neural Network Needs

Another of the many reasons why Torch really shines (glows?) is that it has a comprehensive set of "modules" that you can string together like lego blocks to produce even highly complicated networks. This is largely in part due to the programming abstractions it uses, which we shall soon see.

Like `mnist` we can import the `nn` package using

In [7]:
require 'nn';


### The Real Learning Modules

All of the modules in `nn` are built on a base class known as [`nn.Module`](https://github.com/torch/nn/blob/master/Module.lua). Thus, in order to understand the rest of the package, we must first understand [`nn.Module`](https://github.com/torch/nn/blob/master/Module.lua). The [docs](https://github.com/torch/nn/blob/master/doc/module.md) are comprehensive but somewhat hard to parse, so we'll eschew those and just look at the code:

You'll want to point your browser to the code for [`nn.Module`](https://github.com/torch/nn/blob/master/Module.lua).

Now, direct your attention to the [`init`](https://github.com/torch/nn/blob/master/Module.lua#L3) method. The two important things going on here are the initialization of `self.gradInput` and `self.output`. The former holds the gradient that travels backwards through this layer (think [$\partial f/\partial z$](neural_nets.ipynb#Training)) while the latter holds the output. Practically, this means is that you can access these important values using `myModule.gradInput` and `myModule.output`, respectively. There are some other effects that have to deal with the `_type`, but we'll ignore that for now.

Temporarily skipping over the `parameters` method, we eventually come to [`forward`](https://github.com/torch/nn/blob/master/Module.lua#L25), which just calls `updateOutput`; you generally don't want to change this and, instead, override [`updateOutput`](https://github.com/torch/nn/blob/master/Module.lua#L21).

[`updateOutput`](https://github.com/torch/nn/blob/master/Module.lua#L21) is where (half of) the real work of the Module happens. You override (replace) this function with whatever computation you desire and then make `self.output` the result.

Continuing on, we arrive at [`backward`](https://github.com/torch/nn/blob/master/Module.lua#L29), which is a bit like `forward` in that it simply calls some other methods: `updateGradInput` and `accGradParameters`.

Try your hardest to ignore all of the intervening code and settle your gaze upon [`updateGradInput`](https://github.com/torch/nn/blob/master/Module.lua#L42). This function is the backward counterpart of `updateOutput` which takes the gradient w.r.t. the output (e.g., $\frac{\partial \mathcal{L}}{\partial o}$) and applies the chain rule (e.g., $\frac{\partial \mathcal{L}}{\partial o}\frac{\partial o}{\partial f}$) to get the gradient w.r.t. the input--the output of the previous module. Naturally, this gets stored in `self.gradInput`.

"What about the parameters?" you say. Great question, that's what happens in [`accGradParameters`](https://github.com/torch/nn/blob/master/Module.lua#L46). This function doesn't return anything and, instead, just collects the $\partial z/\partial W$s until we take a gradient descent step at the very end.

Clear as mud, right? No worries, it should become clearer when we get to an example!

#### Mulling Over a Module

Let's scale up the learning with a simple Module, [`nn.Mul`](https://github.com/torch/nn/blob/master/Mul.lua), which learns to multiply the input by a number, $c$. In math parlance, $f(X) = cX$.

On [line 1](https://github.com/torch/nn/blob/master/Mul.lua#L1), you can see that we're creating `nn.Mul` from a base class `nn.Module` (go figure).

In [`__init`](https://github.com/torch/nn/blob/master/Mul.lua#L4) the new thing happening here is that there are two new variables: `self.weight` and `self.gradWeight` which hold the number and the accumulated gradient w.r.t. the number ($\partial\mathcal{L}/dc$). Since this is a `nn.Module` and it calls [`parent.__init(self)`](https://github.com/torch/nn/blob/master/Mul.lua#L4), we know that `self.output` and `self.gradInput` are initialized to empty Tensors.

[`updateOutput`](https://github.com/torch/nn/blob/master/Mul.lua#L23) is reasonably straightforward: it makes some room for the output using `resize` and copies in the input (it's important to do a copy, because it needs the input later to compute the gradient!). After a quick `mul` by $c$, the module returns its output.

[`updateGradInput`](https://github.com/torch/nn/blob/master/Mul.lua#L29) proceeds similarly. This function computes $\partial f/\partial x = c$, and applies the chain rule so that `gradInput = c*gradOutput`. Don't let the call to [`add`](https://torch7.readthedocs.io/en/latest/maths/index.html#res-torchaddres-tensor1-value-tensor2) throw you off: if you check the types (scalar, Tensor) and then look at the [docs](https://torch7.readthedocs.io/en/latest/maths/index.html#res-torchaddres-tensor1-value-tensor2) (scroll down to the first example in red), you'll find that this invocation

> `x:add(value, y)` multiply-accumulates values of `y` into `x`"

Great! Now on to the final piece of the puzzle, [`accGradParameters`](https://github.com/torch/nn/blob/master/Mul.lua#L35), which just takes $\partial f/\partial c = X$, applies the chain rule, and adds the result into the existing `gradWeight`.

In [8]:
mul42 = nn.Mul()
mul42.weight = torch.Tensor{42}
print(mul42)

nn.Mul
{
  weight : DoubleTensor - size: 1
  _type : torch.DoubleTensor
  output : DoubleTensor - empty
  gradInput : DoubleTensor - empty
  gradWeight : DoubleTensor - size: 1
}


In [9]:
input = torch.range(1, 3)
mul42:forward(input)
print(mul42.output)

  42
  84
 126
[torch.DoubleTensor of size 3]



In [10]:
gradOutput = torch.ones(3)
mul42:backward(input, gradOutput)
print(mul42.gradInput)
print(mul42.gradWeight) -- 1 + 2 + 3 = input:sum()

 42
 42
 42
[torch.DoubleTensor of size 3]

 6
[torch.DoubleTensor of size 1]



### So Many Modules
Whereas a vanilla Module is great when you have an input and want to create a new output, most often, you'll want to string other Modules together. For this, you need a [`Container`](https://github.com/torch/nn/blob/master/Container.lua). A [`Container`](https://github.com/torch/nn/blob/master/Container.lua) extends Module to provide, in addition to our friend like `self.output`, `self.gradInput`, `updateOutput()`, and `updateGradInput()`, the `self.modules` property and methods like [`add`](https://github.com/torch/nn/blob/master/Container.lua#L10) and the recursive [`applyToModules`](https://github.com/torch/nn/blob/master/Container.lua#L10).

By far, the most important part of being a Container is that its contained Modules are in the `self.modules` table. Otherwise, the rest of the Container functions won't know to use them!

#### It's as Easy as 1, 2, 3

By far, the most ubiquitous Module is [`nn.Sequential`](https://github.com/torch/nn/blob/master/Sequential.lua), which [takes an input and feeds it--sequentially--through the network](https://github.com/torch/nn/blob/master/Sequential.lua#L41). As this, very much, follows the paradigm of "layers," you'll not be surprised that the wrapper of most of your networks will be `nn.Sequential`.

Here's a quick example that looks a lot like our Perceptron from earlier:

In [11]:
model = nn.Sequential()
model:add(nn.Linear(3, 5))
model:add(nn.ReLU())
model:add(nn.Linear(5, 5))
model:add(nn.ReLU())
model:add(nn.Linear(5, 1))
print(tostring(model)) -- the tostring is only needed in iTorch

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> output]
  (1): nn.Linear(3 -> 5)
  (2): nn.ReLU
  (3): nn.Linear(5 -> 5)
  (4): nn.ReLU
  (5): nn.Linear(5 -> 1)
}	


In [12]:
input = torch.rand(4, 3) -- minibatch of 4 samples with 3 features each
model:forward(input)
for i=1,#model do
    print(model:get(i).output)
end

-0.2110  0.3922 -0.5508  0.3507  0.1708
-0.3264  0.3744 -0.6047  0.2504  0.0523
-0.7199  0.5533 -0.5727  0.2670  0.0941
-0.6018  0.3177 -0.7061  0.0338 -0.3710
[torch.DoubleTensor of size 4x5]

 0.0000  0.3922  0.0000  0.3507  0.1708
 0.0000  0.3744  0.0000  0.2504  0.0523
 0.0000  0.5533  0.0000  0.2670  0.0941
 0.0000  0.3177  0.0000  0.0338  0.0000
[torch.DoubleTensor of size 4x5]

-0.3079  0.2202  0.1640 -0.2994 -0.2039
-0.2506  0.2373  0.1950 -0.2323 -0.2578
-0.2864  0.2186  0.2312 -0.2454 -0.2695
-0.1508  0.2978  0.2428 -0.1306 -0.3454
[torch.DoubleTensor of size 4x5]

 0.0000  0.2202  0.1640  0.0000  0.0000
 0.0000  0.2373  0.1950  0.0000  0.0000
 0.0000  0.2186  0.2312  0.0000  0.0000
 0.0000  0.2978  0.2428  0.0000  0.0000
[torch.DoubleTensor of size 4x5]

 0.1613
 0.1613
 0.1672
 0.1562
[torch.DoubleTensor of size 4x1]



### The Final Judgement

So, you have a model. Cool. Is it any good? Who knows! Actually, the [`Criterion`](https://github.com/torch/nn/blob/master/Criterion.lua) does.

A [`Criterion`](https://github.com/torch/nn/blob/master/Criterion.lua) is kind of like a Module in that it has  `forward` and `backward`, but is unique in that they take a second argument: `target`.

As with Modules, there are a [ton](https://github.com/torch/nn/blob/master/doc/criterion.md) of different Criterions. Choosing the right Criterion is very important as this determines what the network will learn.

### All together now

So now we have a network that takes a vector with three elements and produces a single number. How about we train it to learn how to take the average of the three numbers?

The question of the hour is of what Criterion should we use. One good choice is [`nn.MSECriterion`](https://github.com/torch/nn/blob/master/doc/criterion.md#nn.MSECriterion) which is the $\mathcal{L}(x, y) = (x - y)^2$ we used earlier. This is sort of like asking the output to be "close enough" while ensuring that no value is particularly off.

Here's how the network performs before any training:

In [39]:
origModel = model:clone()
origModel:reset()

testData = torch.range(1, 4*3):view(4, 3)
print(testData)
expectedOutput = testData:mean(2)
print(expectedOutput)

  1   2   3
  4   5   6
  7   8   9
 10  11  12
[torch.DoubleTensor of size 4x3]

  2
  5
  8
 11
[torch.DoubleTensor of size 4x1]



In [40]:
print(origModel:forward(testData))

 0.4441
 0.5053
 0.4998
 0.5056
[torch.DoubleTensor of size 4x1]



Terrible. Absolutely terrible. Let's see if we can get it to learn something!

In [41]:
model:reset()
crit = nn.AbsCriterion() -- (x - y)^2
N = 100000 -- train for 100k iters
for i=1,N do
    local input = torch.Tensor(100, 3):random(10)
    local target = input:max(2)
    model:forward(input)
    local loss = crit:forward(model.output, target)
    
    model:zeroGradParameters()
    crit:backward(model.output, target)
    model:backward(input, crit.gradInput)
    model:updateParameters(0.01)
    
    xlua.progress(i, N)
end

 [======================================================>]                                Tot: 15s452ms | Step: 0ms 

In [42]:
print(model:forward(testData), expectedOutput)

  3.0756
  7.8454
 10.0007
 10.0007
[torch.DoubleTensor of size 4x1]

  2
  5
  8
 11
[torch.DoubleTensor of size 4x1]



Not too shabby for only being able to do sums, multiplications, and thresholding at zero!

## Go Forth (and Back)

You now have all of the tools you need to start classifying MNIST digits! Crazy, right?

### I Want to Be the Very Best...

Start by popping open the [training script](../edit/mnist/main.lua).

You don't have to concern youself much with the details of it but it will help to be aware of a few things:

First is that that there are several named arguments that you can change like `seed`, `nEpochs`, any of the options listed under "Optimization options".

Additionally, on line 39, you'll see that we're using the [`nn.CrossEntropyCriteron`](https://github.com/torch/nn/blob/master/doc/criterion.md#nn.CrossEntropyCriterion) which is like the binary cross-entropy we used in the original Perceptron example but generalized to $n$ classes.

Finally, unlike above where we used `updateParamters(0.01)`, this training script uses the SGD available in the powerful `optim` package which has, along with other optimizers entirely, an implementation of SGD that includes bells and whistles like weight-decay and momentum. This is configured on line 41 and is used on line 75.

### I'm Ready for My Close-Up

Finally, we get to write a real-life model! This model will live in [mnist/models/mlp.lua](../edit/mnist/models/mlp.lua) which is named "mlp.lua" because that's what you're going to write: a multi-layer perceptron!

In the spirit of the max-finding model, above, we can define a model that contains some [`nn.Linear`s](https://github.com/torch/nn/blob/master/doc/simple.md#nn.Linear) and some [non-linearities](https://github.com/torch/nn/blob/master/doc/transfer.md) like [`nn.ReLU`](https://github.com/torch/nn/blob/master/doc/transfer.md#relu). For instance, we will try:




In [14]:
model = nn.Sequential()
model:add(nn.View(26*26))
model:add(nn.Linear(26*26, 1000))
model:add(nn.ReLU())
model:add(nn.Linear(1000, 500))
model:add(nn.ReLU())
model:add(nn.Linear(500, 100))
model:add(nn.ReLU())
model:add(nn.Linear(100, 10))


In [15]:
dofile('mnist/test/mlp_io.lua')

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> output]
  (1): nn.View(784)
  (2): nn.Linear(784 -> 1000)
  (3): nn.ReLU
  (4): nn.Linear(1000 -> 500)
  (5): nn.ReLU
  (6): nn.Linear(500 -> 100)
  (7): nn.ReLU
  (8): nn.Linear(100 -> 10)
}
{
  gradInput : DoubleTensor - empty
  modules : 
    {
      1 : 
        nn.View(784)
        {
          _type : torch.DoubleTensor
          output : DoubleTensor - empty
          gradInput : DoubleTensor - empty
          size : LongStorage - size: 1
          numElements : 784
       

 }
      2 : 
        nn.Linear(784 -> 1000)
        {
          gradBias : DoubleTensor - size: 1000
          weight : DoubleTensor - size: 1000x784
          _type : torch.DoubleTensor
          output : DoubleTensor - empty
          gradInput : DoubleTensor - empty
          bias : DoubleTensor - size: 1000
          gradWeight : DoubleTensor - size: 1000x784
        }
      3 : 
        nn.ReLU
        {
          inplace : false
          threshold : 0
          _type : torch.DoubleTensor
          output : DoubleTensor - empty
          gradInput : DoubleTensor - empty
          val : 0
        }
      4 : 
        nn.Linear(1000 -> 500)
        {
          gradBias : DoubleTensor - size: 500
          weight : DoubleTensor - size: 500x1000
          _type : torch.DoubleTensor
          output : DoubleTensor - empty
          gradInput : DoubleTensor - empty
          bias : DoubleTensor - size: 500
          gradWeight : DoubleTensor - size: 500x1000
        }
      5 : 
     

Passed!	



When you're ready to roll, run the following code block. Try to get the best performance you can!

In [16]:
trainMNIST = dofile('mnist/main.lua')

trainMNIST()
-- trainMNIST({dispFreq=2, nEpochs=10, batchSize=1024, lr=0.01}) -- twiddle some hyperparameters

 [=====================================================================================>]                                Tot: 30ms | Step: 3ms 

Epoch  1 | train loss: 0.290 | val loss: 0.137 | val acc: 95.86%	


Epoch  2 | train loss: 0.095 | val loss: 0.097 | val acc: 96.98%	


 [=====================================================================================>]                                Tot: 43ms | Step: 4ms 

Epoch  3 | train loss: 0.058 | val loss: 0.086 | val acc: 97.30%	


...

Epoch 18 | train loss: 0.001 | val loss: 0.079 | val acc: 98.04%	


 [=====================================================================================>]                                Tot: 40ms | Step: 4ms 

Epoch 19 | train loss: 0.001 | val loss: 0.080 | val acc: 98.03%	


 [=====================================================================================>]                                Tot: 33ms | Step: 3ms 

Epoch 20 | train loss: 0.001 | val loss: 0.080 | val acc: 98.03%	
